In [0]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib


In [0]:
# Define the path to the Delta table
delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"

# Read the Delta table to a Spark DataFrame
aws_keys_df = spark.read.format("delta").load(delta_table_path)

# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

# AWS S3 bucket name
AWS_S3_BUCKET = "user-1279c94681db-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/saiful_bucket"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)


--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-3819816579781016> in <module> 
 18 SOURCE_URL = "s3n://{0}:{1}@{2}" . format ( ACCESS_KEY , ENCODED_SECRET_KEY , AWS_S3_BUCKET ) 
 19 # Mount the drive 
 ---> 20 dbutils . fs . mount ( SOURCE_URL , MOUNT_NAME ) 
 21 
 22 display ( dbutils . fs . ls ( "/mnt/saiful_bucket" ) ) 

 /databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 387 exc . __context__ = None 
 388 exc . __cause__ = None 
 --> 389 raise exc
 390 
 391 return f_with_exception_handling

 ExecutionError : An error occurred while calling o7025.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/saiful_bucket; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/saiful_bucket
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:128)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.createOrUpdateMount(DBUtilsCore.scala:756)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:776)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:750)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/saiful_bucket
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$insertMount$1(MetadataManager.scala:704)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$modifyAndVerify$2(MetadataManager.scala:1080)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:853)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.modifyAndVerify(MetadataManager.scala:1069)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.insertMount(MetadataManager.scala:712)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:128)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:54)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:53)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:53)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$4.$anonfun$applyOrElse$10(DbfsServerBackend.scala:407)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:435)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.AttributionContext$.withValue(AttributionContext.scala:216)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:433)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:427)
	at com.databricks.rpc.ServerBackend.withAttributionContext(ServerBackend.scala:22)
	at com.databricks.loggin

In [0]:
display(dbutils.fs.ls("/mnt/saiful_bucket"))

path,name,size,modificationTime
dbfs:/mnt/saiful_bucket/kafka-connect-s3/,kafka-connect-s3/,0,1711366815271
dbfs:/mnt/saiful_bucket/topics/,topics/,0,1711366815271


In [0]:
%sql
-- Disable format checks during reading of the Delta tables
SET spark.databricks.delta.formatCheck.enabled=false

key,value
spark.databricks.delta.formatCheck.enabled,false


In [0]:
# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location_1 = "/mnt/saiful_bucket/topics/1279c94681db.pin/partition=0/*.json"
file_location_2 = "/mnt/saiful_bucket/topics/1279c94681db.geo/partition=0/*.json"
file_location_3 = "/mnt/saiful_bucket/topics/1279c94681db.user/partition=0/*.json"
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
df_pin = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location_1)
df_geo = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location_2)
df_user = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location_3)
# Display Spark dataframe to check its content
display(df_pin)
display(df_geo)
display(df_user)

category,description,downloaded,follower_count,image_src,index,is_image_or_video,poster_name,save_location,tag_list,title,unique_id
travel,"This Costa Rica itinerary is the ultimate guide to spending two weeks in Costa Rica. Find out about visiting La Fortuna, Arenal, Monteverde, Naranjo, Corcovado National Park, Or…",1,10k,https://i.pinimg.com/originals/30/93/cb/3093cb01d9de2d125fda8ba5e3e41946.jpg,10138,image,"Wanderlust Chloe ✈️ Travel guides, inspo and adventure travel ✈️",Local save in /data/travel,"Costa Rica Travel,Rio Celeste Costa Rica,Dream Vacations,Vacation Spots,Vacation Travel,Travel Pictures,Travel Photos,Fortuna Costa Rica,Costa Rica Pictures","14 Amazing Things To Do In Costa Rica | Volcanoes, Waterfalls, Wildlife And More",927c4658-cc3f-4b92-9b5c-70743d0c238d
diy-and-crafts,"This post may contain affiliate links, read our Disclosure Policy for more information. As an Amazon Associate I earn from qualifying purchases, thank you! Make some cute handpr…",1,892k,https://i.pinimg.com/originals/ff/fe/38/fffe384f3ec18a0d87cb2d80cc8c1499.jpg,3156,image,Michelle {CraftyMorning.com},Local save in /data/diy-and-crafts,"Christmas Gifts For Parents,Christmas Decorations For Kids,Christmas Crafts For Toddlers,Preschool Christmas,Christmas Crafts For Gifts,Christmas Activities,Toddler Crafts,Kids Christmas,Christmas Feeling",Handprint Reindeer Ornaments - Crafty Morning,fa6e31a4-18c2-4eca-a6d8-e903eee2c2a4
finance,"If you love budgeting, make sure to give Dave Ramsey's 7 Baby Steps a try. Follow these steps to begin your debt snowball, build an emergency fund, invest and reach riches. I ca…",1,26k,https://i.pinimg.com/originals/1e/9d/90/1e9d906e4e150e3b95187f3b76ea7c71.png,5494,image,"Living Low Key | Save Money, Make Money, & Frugal Living",Local save in /data/finance,"Financial Peace,Financial Tips,Saving Money Quotes,Total Money Makeover,Budgeting Finances,Money Management,Wealth Management,Personal Finance,Making Ideas",Dave Ramsey's 7 Baby Steps: What Are They And Will They Work For You,8fb2af68-543b-4639-8119-de33d28706ed
finance,"If you love budgeting, make sure to give Dave Ramsey's 7 Baby Steps a try. Follow these steps to begin your debt snowball, build an emergency fund, invest and reach riches. I ca…",1,26k,https://i.pinimg.com/originals/1e/9d/90/1e9d906e4e150e3b95187f3b76ea7c71.png,5494,image,"Living Low Key | Save Money, Make Money, & Frugal Living",Local save in /data/finance,"Financial Peace,Financial Tips,Saving Money Quotes,Total Money Makeover,Budgeting Finances,Money Management,Wealth Management,Personal Finance,Making Ideas",Dave Ramsey's 7 Baby Steps: What Are They And Will They Work For You,8fb2af68-543b-4639-8119-de33d28706ed
finance,"If you love budgeting, make sure to give Dave Ramsey's 7 Baby Steps a try. Follow these steps to begin your debt snowball, build an emergency fund, invest and reach riches. I ca…",1,26k,https://i.pinimg.com/originals/1e/9d/90/1e9d906e4e150e3b95187f3b76ea7c71.png,5494,image,"Living Low Key | Save Money, Make Money, & Frugal Living",Local save in /data/finance,"Financial Peace,Financial Tips,Saving Money Quotes,Total Money Makeover,Budgeting Finances,Money Management,Wealth Management,Personal Finance,Making Ideas",Dave Ramsey's 7 Baby Steps: What Are They And Will They Work For You,8fb2af68-543b-4639-8119-de33d28706ed
finance,"If you love budgeting, make sure to give Dave Ramsey's 7 Baby Steps a try. Follow these steps to begin your debt snowball, build an emergency fund, invest and reach riches. I ca…",1,26k,https://i.pinimg.com/originals/1e/9d/90/1e9d906e4e150e3b95187f3b76ea7c71.png,5494,image,"Living Low Key | Save Money, Make Money, & Frugal Living",Local save in /data/finance,"Financial Peace,Financial Tips,Saving Money Quotes,Total Money Makeover,Budgeting Finances,Money Management,Wealth Management,Personal Finance,Making Ideas",Dave Ramsey's 7 Baby Steps: What Are They And Will They Work For You,8fb2af68-543b-4639-8119-de33d28706ed
christmas,Here are the best DIY

country,ind,latitude,longitude,timestamp
Antarctica (the territory South of 60 deg S),2418,-88.4642,-171.061,2022-05-27T11:30:59
Cocos (Keeling) Islands,10794,-89.5236,-154.567,2022-01-01T02:26:50
Cocos (Keeling) Islands,10794,-89.5236,-154.567,2022-01-01T02:26:50
Cocos (Keeling) Islands,10794,-89.5236,-154.567,2022-01-01T02:26:50
Central African Republic,2074,-52.3213,-50.11,2019-11-03T05:41:59
Dominican Republic,9979,14.9967,-120.682,2018-07-18T19:01:46
Cote d'Ivoire,2923,-84.6302,-164.507,2019-09-08T22:53:09
Cote d'Ivoire,2923,-84.6302,-164.507,2019-09-08T22:53:09
French Guiana,8304,-28.8852,-164.87,2019-09-13T04:50:29
Cote d'Ivoire,4315,-45.8508,66.1003,2019-12-15T03:51:28


age,date_joined,first_name,ind,last_name
21,2015-11-10T09:27:42,Andrea,8731,Alexander
21,2015-11-10T09:27:42,Andrea,8731,Alexander
21,2015-11-10T09:27:42,Andrea,8731,Alexander
36,2015-12-20T16:38:13,Michelle,4315,Prince
36,2015-12-20T16:38:13,Michelle,4315,Prince
32,2017-10-10T20:09:33,Christian,10625,Lang
36,2015-12-20T16:38:13,Michelle,4315,Prince
32,2016-04-02T03:51:23,Brittany,1313,Jones
34,2016-12-22T00:02:02,Thomas,10794,Turner
32,2016-04-02T03:51:23,Brittany,1313,Jones
